In [ ]:
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
import datetime as dt
from autogen import AssistantAgent
from inswitch.llm.model import get_openai_model_config
import time
import requests
import uuid
from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent

app = Flask(__name__)
CORS(app)

llm_config = {"config_list": [get_openai_model_config()]}
assistant = AssistantAgent(name="assistant", llm_config=llm_config)

In [ ]:
knowledge_asker = get_chat_agent(
    name = "knowledge-asker",
    system_message="You will get an intent from the stakeholders on how to manage the system."
        "You need to figure out what knowledge about the system do you need."
        "You should reply to describe what knowledge you need, and that's all. Do not make any"
        "assumptions or decision, as you haven't seen the knowledge."
)

knowledge_extractor = get_chat_agent(
    name = "knowledge_extractor",
    system_message="""
        You will receive a transcript of message exchanges between multiple autogen agents.

        Your job is to generate a clear and structured summary of what each agent did or said,
        without interpretation, rephrasing, or guessing intentions.

        For each agent:
        - Indicate the agent name.
        - Indicate their role if known (e.g., human_proxy, knowledge_asker).
        - List clearly what they said (verbatim if possible), and in what context.

        Do not summarize the user's intent.
        Do not paraphrase or analyze.
        Do not infer meaning or goals beyond what was said.

        Stick strictly to the content of the transcript.
        Format your output cleanly as:
        Agent: [name]
        Role: [role]
        Messages:
        - [message 1]
        - [message 2]
        ...
    """
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)

In [ ]:
# REPOSITORY
# Handles access to the database

def fetch_intents_from_graphdb():
    """
    Function used to get all of the intent stored in the database.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = """
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?id ?authorName ?content ?creationDate ?lastUpdate
    WHERE {
        ?intent a icm:Intent ;
        ex:author ?author ;
        ex:content ?content ;
        dcterms:created ?creationDate ;
        dcterms:modified ?lastUpdate .
        ?author foaf:name ?authorName .
        BIND(STR(?intent) AS ?id)
    }
    ORDER BY ASC(?creationDate)
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data={"query": sparql_query},
        headers={"Accept": "application/sparql-results+json"}
    )
    
    return response


def insert_intent_into_graphdb(intent_id : str, author : str, content : str, creation_date : str):
    """
    Function used to insert a new intent in the database.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    author : str
        The author of the intent.
    content : str
        The content of the intent.
    creation_date : str
        The date the intent has been made.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {{
        ex:{intent_id} a icm:Intent ;
            ex:author <http://example.com/intent/person/{author}> ;
            ex:content "{content}" ;
            dcterms:created "{creation_date}"^^xsd:dateTime ;
            dcterms:modified "{creation_date}"^^xsd:dateTime .

        <http://example.com/intent/person/{author}> a foaf:Person ;
            foaf:name "{author}" .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query,
        headers={"Content-Type": "application/sparql-update"}
    )

    return response

def insert_report_into_graphdb(report_id : str, intent_id : str, agent_name : str, agent_response : str, response_time : str, creation_date : str):
    """
    Function used to insert a new intent report in the database.

    Parameters
    ----------
    report_id : str
        The ID of the intent report.
    intent_id : str
        The ID of the intent.
    agent_name : str
        The name of the agent who handled the intent.
    agent_response : str
        The response that gave the agent.
    response_time : str
        The execution time of the handle of the intent.
    creation_date : str
        The creation date of the intent report.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> a icm:IntentReport ;
            dcterms:created "{creation_date}"^^xsd:dateTime ;
            ex:agentName "{agent_name}" ;
            ex:executionTime "{response_time}"^^xsd:float ;
            ex:intent <http://example.com/intent/{intent_id}> ;
            ex:response "{agent_response}" .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response

def select_intent_from_graphdb(intent_id : str):
    """
    Function use to get an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """
    
    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?id ?authorName ?content ?creationDate ?lastUpdate
    WHERE {{
        BIND(<http://example.com/intent/{intent_id}> AS ?intent)
        ?intent a icm:Intent ;
        ex:author ?author ;
        ex:content ?content ;
        dcterms:created ?creationDate ;
        dcterms:modified ?lastUpdate .
        ?author foaf:name ?authorName .
        BIND(STR(?intent) AS ?id)
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )
    
    return response

def fetch_intent_reports_of_intent_from_graphdb(intent_id : str):
    """
    Function used to get all of the intent reports of an intent, stored in the database.
    
    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """
    
    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT ?report ?creationDate ?agentName ?response ?executionTime
    WHERE {{
        ?report a icm:IntentReport ;
                dcterms:created ?creationDate ;
                ex:agentName ?agentName ;
                ex:response ?response ;
                ex:executionTime ?executionTime ;
                ex:intent <http://example.com/intent/{intent_id}> .
    }}
    ORDER BY ASC(?creationDate)
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )

    return response

def fetch_single_intent_report_from_graphdb(intent_id : str, report_id : str):
    """
    Function use to get an intent report of an intent, from their IDs.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX icm: <http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT ?report ?creationDate ?agentName ?response ?executionTime
    WHERE {{
        BIND(<http://example.com/intent/{intent_id}/intentReport/{report_id}> AS ?report)
        ?report a icm:IntentReport ;
            dcterms:created ?creationDate ;
            ex:agentName ?agentName ;
            ex:response ?response ;
            ex:executionTime ?executionTime ;
            ex:intent <http://example.com/intent/{intent_id}> .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db",
        data=sparql_query.encode("utf-8"),
        headers={
            "Content-Type": "application/sparql-query",
            "Accept": "application/sparql-results+json"
        }
    )

    return response


def update_intent_content_from_graphdb(intent_id : str, new_content : str, update_date : str):
    """
    Function used to update the content of an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    new_content : str
        The new content of the intent.
    update_date : str
        The date the intent has been updated.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    
    DELETE {{
        <http://example.com/intent/{intent_id}> ex:content ?oldContent .
        <http://example.com/intent/{intent_id}> dcterms:modified ?oldDate .
    }}
    INSERT {{
        <http://example.com/intent/{intent_id}> ex:content "{new_content}" .
        <http://example.com/intent/{intent_id}> dcterms:modified "{update_date}"^^xsd:dateTime .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}> ex:content ?oldContent .
        OPTIONAL {{ <http://example.com/intent/{intent_id}> dcterms:modified ?oldDate . }}
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response


def delete_intent_from_graphdb(intent_id : str):
    """
    Function used to delete an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    DELETE {{
        <http://example.com/intent/{intent_id}> ?p ?o .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}> ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def delete_reports_of_intent_from_graphdb(intent_id : str):
    """
    Function used to delete all intent reports of an intent.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """

    # QUERY
    sparql_query = f"""
    PREFIX ex: <http://example.com/intent/>

    DELETE {{
        ?report ?p ?o .
    }}
    WHERE {{
        ?report ex:intent <http://example.com/intent/{intent_id}> .
        ?report ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )
    
    return response

def delete_report_of_intent_from_graphdb(intent_id : str, report_id : str):
    """
    Function used to delete an intent report of an intent, from their IDs.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.

    Returns
    -------
    requests.Response
        Response object containing the results of the SPARQL query in JSON format.
    """
    
    # QUERY
    sparql_query = f"""
    DELETE {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> ?p ?o .
    }}
    WHERE {{
        <http://example.com/intent/{intent_id}/intentReport/{report_id}> ?p ?o .
    }}
    """

    response = requests.post(
        "http://graphdb:7200/repositories/intent-db/statements",
        data=sparql_query.encode("utf-8"),
        headers={"Content-Type": "application/sparql-update"}
    )

    return response


In [ ]:
# SERVICE
# Contains logic

def get_all_intents():
    """
    Function used to analyse the response of the database to get all the intents stored and
    to format them.
    
    Returns
    -------
    intents : List
        List of intents.
    """

    response = fetch_intents_from_graphdb()

    if response.status_code != 200:
        return None
    
    intents_db = response.json()["results"]["bindings"]

    if not intents_db:
        return None

    intents = []

    for i in intents_db:
        id = i['id']["value"].split("/")[-1]

        intent = {
            "id": id,
            "author": i["authorName"]["value"],
            "content": i["content"]["value"],
            "creationDate": i["creationDate"]["value"],
            "lastUpdateDate": i["lastUpdate"]["value"]
        }
        intents.append(intent)
    
    return intents

def get_all_intents_json_ld():
    """
    Function used to format the list of intent in JSON-LD format.
    
    Returns
    -------
    jsonld_data : List
        List of intents in JSON-LD format.
    """
    
    intents = get_all_intents()
    
    if not intents:
        return None
    
    # Definition of the context
    jsonld_data = {
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema#"
        }
    }
    jsonld_data["intents"] = []

    for intent in intents:
        id = intent['id'].split("/")[-1]
        
        creation_date_obj = dt.datetime.fromisoformat(intent['creationDate'])
        update_date_obj = dt.datetime.fromisoformat(intent['lastUpdateDate'])

        intent_data = {
            "@id": f"ex:{id}",
            "@type": "icm:Intent",
            "ex:author": {
                "@id": f"ex:person/{intent['author'].replace(' ', '_')}",
                "@type": "foaf:Person",
                "foaf:name": intent['author']
            },
            "ex:content": intent['content'],
            "dcterms:created": {
                "@value": creation_date_obj.isoformat(),
                "@type": "xsd:dateTime"
            },
            "dcterms:modified": {
                "@value": update_date_obj.isoformat(),
                "@type": "xsd:dateTime"
            }
        }
        jsonld_data["intents"].append(intent_data)

    return jsonld_data

def select_intent_from_id(intent_id : str):
    """
    Function used to analyse the response of the database to get an intent from its ID and
    to format it.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    
    Returns
    -------
    intent : Dictionary
        The intent with all its attributes.
    """
    
    response = select_intent_from_graphdb(intent_id)

    if response.status_code != 200:
        return None

    intent_db = response.json()["results"]["bindings"][0]

    if not intent_db:
        return None

    id = intent_db['id']["value"].split("/")[-1]

    intent = {
        "id": id,
        "author": intent_db["authorName"]["value"],
        "content": intent_db["content"]["value"],
        "creationDate": intent_db["creationDate"]["value"],
        "lastUpdateDate": intent_db["lastUpdate"]["value"]
    }
    
    return intent

def select_intent_json_ld_from_id(intent_id : str):
    """
    Function used to format an intent in JSON-LD format.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    
    Returns
    -------
    jsonld_data : Dictionary
        The intent with all its attributes in JSON-LD format.
    """
    
    response = select_intent_from_graphdb(intent_id)

    if response.status_code != 200:
        return None

    intent_db = response.json()["results"]["bindings"][0]

    if not intent_db:
        return None

    id = intent_db['id']["value"].split("/")[-1]
    
    creation_date_obj = dt.datetime.fromisoformat(intent_db["creationDate"]["value"])
    update_date_obj = dt.datetime.fromisoformat(intent_db["lastUpdate"]["value"])

    jsonld_data = {
        # Definition of the context
        "@context": {
            "ex": "http://example.com/intent/",
            "icm": "http://tio.models.tmforum.org/tio/v3.6.0/IntentCommonModel/",
            "foaf": "http://xmlns.com/foaf/0.1/",
            "dcterms": "http://purl.org/dc/terms/",
            "xsd": "http://www.w3.org/2001/XMLSchema#"
        },
        "@id": f"ex:{id}",
        "@type": "icm:Intent",
        "ex:author": {
            "@id": f"ex:person/{intent_db["authorName"]["value"].replace(' ', '_')}",
            "@type": "foaf:Person",
            "foaf:name": intent_db["authorName"]["value"]
        },
        "ex:content": intent_db["content"]["value"],
        "dcterms:created": {
            "@value": creation_date_obj.isoformat(),
            "@type": "xsd:dateTime"
        },
        "dcterms:modified": {
            "@value": update_date_obj.isoformat(),
            "@type": "xsd:dateTime"
        }
    }
    
    return jsonld_data

def get_all_intent_reports_of_intent(intent_id : str):
    """
    Function used to analyse the response of the database to get all the intent reports 
    of an intent from its ID and to format them.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    
    Returns
    -------
    intent_reports : List
        The list of all of the intent reports and their attributes.
    """
    
    if not select_intent_from_id(intent_id):
        return None
    
    response = fetch_intent_reports_of_intent_from_graphdb(intent_id)

    if response.status_code != 200:
        return None

    results = response.json()["results"]["bindings"]

    if not results:
        return None

    intent_reports = []

    for r in results:
        id = r['report']["value"].split("/")[-1]

        intent_report = {
            "id": id,
            "agentName": r["agentName"]["value"],
            "response": r["response"]["value"],
            "creationDate": r["creationDate"]["value"],
            "executionTime": r["executionTime"]["value"]
        }
        intent_reports.append(intent_report)
    
    return intent_reports
    
def get_intent_report_of_intent(intent_id : str, report_id : str):
    """
    Function used to analyse the response of the database to get an intent report
    of an intent from their IDs and to format it.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.
    
    Returns
    -------
    intent_report : Dictionnary
        An intent report and its attributes.
    """
    
    if not select_intent_from_id(intent_id):
        return None
    
    response = fetch_single_intent_report_from_graphdb(intent_id, report_id)

    if response.status_code != 200:
        return None

    result = response.json()["results"]["bindings"]

    if not result:
        return None

    intent_report_data = result[0]

    id = result[0]['report']["value"].split("/")[-1]

    intent_report = {
        "id": id,
        "agentName": intent_report_data["agentName"]["value"],
        "response": intent_report_data["response"]["value"],
        "creationDate": intent_report_data["creationDate"]["value"],
        "executionTime": intent_report_data["executionTime"]["value"]
    }
    
    return intent_report


def create_intent(author : str, content : str):
    """
    Function used to create a new intent and its associated intent report.

    Parameters
    ----------
    author : str
        The author of the intent.
    content : str
        The content of the intent.
    
    Returns
    -------
    Dictionnary
        A dictionnary containing the intent and its intent report.
    """
    
    intent_id = uuid.uuid4()
    creation_date = dt.datetime.now(dt.timezone.utc).isoformat()

    # Cleaning of the author name
    author_without_space = author.replace(" ", "_")
    
    response = insert_intent_into_graphdb(intent_id, author_without_space, content, creation_date)
    
    if response.status_code not in {200, 201, 202, 204}:
        return None

    intent = select_intent_from_id(intent_id)
    
    report = create_intent_report(intent_id, content)

    if not intent or not report:
        return None
    
    return { "intent": intent, "intent_report": report }

def create_intent_report(intent_id : str, content : str):
    """
    Function used to call the agent with the content of the author and create an intent
    report with the response.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    content : str
        The content of the intent.

    Returns
    -------
    intent_report : Dictionnary
        An intent report and its attributes.
    """
    startTime = time.time()

    report_id_response = uuid.uuid4()
    creation_date = dt.datetime.now(dt.timezone.utc).isoformat()

    chat_result = moderator.initiate_chat(
        recipient= knowledge_asker,
        message= "here is the intent : " + content,
        max_turns= 1,
        summary_method= "last_msg"
    )

    agent_response = chat_result.summary

    # Cleaning the response of the agent
    agent_response = agent_response.replace('\\', '\\\\')
    agent_response = agent_response.replace('"', '\\"')
    agent_response = agent_response.replace('\n', '\\n')

    response = insert_report_into_graphdb(report_id_response, intent_id, knowledge_asker.name, agent_response, (time.time() - startTime), creation_date)

    if response.status_code not in {200, 201, 202, 204}:
        return None
    
    startTime = time.time()

    report_id_summary = uuid.uuid4()
    creation_date = dt.datetime.now(dt.timezone.utc).isoformat()

    chat_result_extracted = moderator.initiate_chat(
        recipient= knowledge_extractor,
        message= repr(chat_result),
        max_turns= 1,
        summary_method= "last_msg"
    )

    summary = chat_result_extracted.summary

    # Cleaning the summary of the agent
    summary = summary.replace('\\', '\\\\')
    summary = summary.replace('"', '\\"')
    summary = summary.replace('\n', '\\n')
    
    response = insert_report_into_graphdb(report_id_summary, intent_id, knowledge_extractor.name, summary, (time.time() - startTime), creation_date)

    if response.status_code not in {200, 201, 202, 204}:
        return None

    intent_report_response = get_intent_report_of_intent(intent_id, report_id_response)
    intent_report_summary = get_intent_report_of_intent(intent_id, report_id_summary)

    intent_report = {
        "response": intent_report_response,
        "summary": intent_report_summary
    }

    return intent_report

def update_partially_intent(intent_id : str, content : str):
    """
    Function used to update the content of an existing intent and create a new intent report.

    Parameters
    ----------
    author : str
        The author of the intent.
    content : str
        The content of the intent.
    
    Returns
    -------
    Dictionnary
        A dictionnary containing the modified intent and its new intent report.
    """
    if not select_intent_from_id(intent_id):
        return None
    
    update_date = dt.datetime.now(dt.timezone.utc).isoformat()
    
    update_response = update_intent_content_from_graphdb(intent_id, content, update_date)

    if update_response.status_code not in {200, 201, 202, 204}:
        return None
    
    intent_report = create_intent_report(intent_id, content)

    if not intent_report:
        return None
    
    intent = select_intent_from_id(intent_id)
    
    return { "intent": intent, "intent_report": intent_report }

def delete_intent_from_id(intent_id : str):
    """
    Function used to delete an intent from its ID and all its intent reports.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    bool
        Statement that the intent has been deleted.
    """
    if not select_intent_from_id(intent_id):
        return False
    
    delete_reports_response = delete_reports_of_intent_from_graphdb(intent_id)

    if delete_reports_response.status_code not in {200, 201, 202, 204}:
        return False
    
    delete_intent_response = delete_intent_from_graphdb(intent_id)

    if delete_intent_response.status_code not in {200, 201, 202, 204}:
        return False
    
    return True

def delete_report_from_id(intent_id : str, report_id : str):
    """
    Function used to delete an intent report from its ID and the intent ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.

    Returns
    -------
    bool
        Statement that the intent report has been deleted.
    """
    if not select_intent_from_id(intent_id):
        return False
    
    if not get_intent_report_of_intent(intent_id, report_id):
        return False
    
    response = delete_report_of_intent_from_graphdb(intent_id, report_id)

    if response.status_code not in {200, 201, 202, 204}:
        return False
    
    return True
    


In [ ]:
# CONTROLLER
# Handles HTTP requests

# GET METHODS

@app.get("/")
def get_homepage():
    """
    Homepage displaying available API endpoints.

    Returns
    -------
    Response
        Rendered HTML template for the homepage.
    int
        HTTP status code.
    """
    return render_template('index.html')

@app.get("/intent")
def get_intents():
    """
    Function used to retrieve the list of intent.

    Returns
    -------
    JSON
        List of intents and their attributes.
    int
        HTTP status code.
    """
    try:
        intents = get_all_intents()
        if intents:
            return jsonify(intents), 200
        else:
            return '', 204
    except Exception as e:
        return str(e), 500 

@app.get("/intent/json-ld")
def get_intents_json_ld():
    """
    Function used to retrieve the list of intent as RDF in JSON-LD format.

    Returns
    -------
    JSON
        List of intents and their attributes as RDF in JSON-LD format.
    int
        HTTP status code.
    """
    try:
        intents = get_all_intents_json_ld()
        if intents:
            return jsonify(intents), 200
        else:
            return '', 204
    except Exception as e:
        return str(e), 500

@app.get("/intent/<string:intent_id>")
def get_intent_from_id(intent_id : str):
    """
    Function used to retrieve an intent from its ID.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    JSON
        An intent and its attributes.
    int
        HTTP status code.
    """
    try:
        intent = select_intent_from_id(intent_id)
        if intent:
            return jsonify(intent), 200
        else:
            return '', 204
    except Exception as e:
        return str(e), 500

@app.get("/intent/<string:intent_id>/json-ld")
def get_intent_json_ld_from_id(intent_id : str):
    """
    Function used to retrieve an intent from its ID as RDF in JSON-LD format.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    JSON
        An intent and its attributes as RDF in JSON-LD format.
    int
        HTTP status code.
    """
    try:
        intent = select_intent_json_ld_from_id(intent_id)
        if intent:
            return jsonify(intent), 200
        else:
            return '', 204
    except Exception as e:
        return str(e), 500
    

# Intent Report

@app.get("/intent/<string:intent_id>/intentReport")
def get_intent_report_of_intent_id(intent_id : str):
    """
    Function used to retrieve the list of intent reports of an intent.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    JSON
        List of intent reports and their attributes.
    int
        HTTP status code.
    """
    try:
        intent_reports = get_all_intent_reports_of_intent(intent_id)
        if intent_reports:
            return jsonify(intent_reports), 200
        else:
            return '', 204
    except Exception as e:
        return str(e), 500

@app.get("/intent/<string:intent_id>/intentReport/<string:report_id>")
def get_intent_report_from_id_of_intent_id(intent_id : str, report_id : str):
    """
    Function used to retrieve an intent report of an intent from their IDs.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id :
        The ID of the intent report.

    Returns
    -------
    JSON
        An intent and its attributes.
    int
        HTTP status code.
    """
    try:
        intent_report = get_intent_report_of_intent(intent_id, report_id)
        if intent_report:
            return jsonify(intent_report), 200
        else:
            return '', 204
    except Exception as e:
        return str(e), 500


# POST METHODS

@app.post("/intent")
def add_intent():
    """
    Function used to create a new intent.

    Returns
    -------
    JSON
        The intent with its intent report.
    int
        HTTP status code.
    """
    if request.is_json:
        try:
            added_intent = request.get_json()

            if "author" in added_intent and added_intent["author"] != "" and "content" in added_intent and added_intent["content"] != "":
                intent_and_report = create_intent(added_intent["author"], added_intent["content"])

                if not intent_and_report:
                    return {"error": "Error during adding a new intent"}, 415

                return jsonify(intent_and_report), 201
            else:
                return {"error": "The JSON request is not correct"}, 415
        except Exception as e:
            return str(e), 500
    else:
        return {"error": "Request must be JSON"}, 415
    

# PATCH METHODS

@app.patch("/intent/<string:intent_id>")
def update_intent(intent_id : str):
    """
    Function used to update an existing intent.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    JSON
        The modified intent and its new intent report.
    int
        HTTP status code.
    """
    if request.is_json:
        updated_intent = request.get_json()

        if "content" in updated_intent and updated_intent["content"] != "":
            try:
                intent_and_report = update_partially_intent(intent_id, updated_intent["content"])

                if not intent_and_report:
                    return {"error": "Error during updating the intent"}, 415

                return intent_and_report, 201
            except Exception as e:
                return str(e), 500
        else:
            return {"error": "The JSON request is not correct"}, 415
    else:
        return {"error": "Request must be JSON"}, 415
        

# DELETE METHODS

@app.delete("/intent/<string:intent_id>")
def delete_intent(intent_id : str):
    """
    Function used to delete an existing intent.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.

    Returns
    -------
    JSON
        Message of deletion.
    int
        HTTP status code.
    """
    try:
        response = delete_intent_from_id(intent_id)
        if response:
            return jsonify({"message": "Intent deleted successfully"}), 200
        else:
            return jsonify({"error": "Error during the deletion of the intent."}), 500
    except Exception as e:
        return str(e), 500

@app.delete("/intent/<string:intent_id>/intentReport/<string:report_id>")
def delete_intent_report_from_intent(intent_id : str, report_id : str):
    """
    Function used to delete an existing intent report of an intent.

    Parameters
    ----------
    intent_id : str
        The ID of the intent.
    report_id : str
        The ID of the intent report.

    Returns
    -------
    JSON
        Message of deletion.
    int
        HTTP status code.
    """
    try:
        response = delete_report_from_id(intent_id, report_id)
        if response:
            return jsonify({"message": "Intent report deleted successfully"}), 200
        else:
            return jsonify({"error": "Error during the deletion of the intent report."}), 500
    except Exception as e:
        return str(e), 500



In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)